In [1]:
import logging
import config

from IPython import display
from dataset.TPL import *
import dataset.preprocessing as pre

import numpy as np
import pandas as pd
import time
import pickle
import ftn
# from pprint import pprint
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.sklearn

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

%matplotlib inline
%reload_ext autoreload
%autoreload 2
logging.basicConfig(level=logging.INFO)

### 5) Resultados:
Nuestro objetivo ahora es concatenar el dataframe que nos muestra la contribución de los tópicos para cada una de las traces del dataset original al 'dataset'.index. Para ello creemos primero el dataframe topic-trace (4.2):

In [4]:
#Importamos el dataset completo preprocesado:
gravity_SES=pickle.load(open("gravity_SES","rb"))

In [10]:
gravity_SES

0      ['Started_at_{}_(underlined)', 'GRAVITY_dual_o...
1      ['Started_at_{}_(underlined)', 'GRAVITY_dual_o...
2      ['GRAVITY_dual_obs_exp_--_Dual_Field_Science_(...
3      ['Started_at_{}_(underlined)', 'GRAVITY_dual_o...
4      ['GRAVITY_dual_obs_exp_--_Dual_Field_Science_(...
                             ...                        
987    ['GRAVITY_gen_tec_AcqFibA_--_Calibrate_AcqCam_...
988    ['Started_at_{}_(underlined)', 'GRAVITY_gen_te...
989    ['Started_at_{}_(underlined)', 'GRAVITY_gen_te...
990    ['Started_at_{}_(underlined)', 'GRAVITY_gen_te...
991    ['GRAVITY_gen_tec_SaveDb_--_Save_Instrument_Da...
Length: 992, dtype: object

In [5]:
#Recreamos nuestro modelo más óptimo:
bestmodel_gravity=pickle.load(open("gravity_bestmodel","rb"))
gravity_train=pickle.load(open("gravity_train","rb"))
#Le enseñamos el vocabulario a la clase 'v' CountVectorizer() a través de la matriz token-frequency
info_train_v, v= ftn.vectorizer(gravity_train)

In [8]:
#Verificamos que todo esté en orden:

#le entregamos nuestra matriz token-frequency creada anteriormente
bestmodel_gravity.fit(info_train_v)

# making LDA TOP MATRIX USING CORPUS TF
lda_topic_modelling = bestmodel_gravity.fit_transform(info_train_v)

In [7]:
#Dejamos todo como str (la info contiene números)
gravity_SES=gravity_SES.apply(str)

#Aplicamos el CV
gravity_SES_v = v.transform(gravity_SES)

In [9]:
#Mostramos la contribución de los tópicos a cada una de las traces:
df_topic_dc=ftn.topic_document(bestmodel_gravity,gravity_SES_v)
df_topic_dc

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000046,0.963814,0.035722,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046
1,0.029996,0.000059,0.000059,0.000059,0.000059,0.901173,0.000059,0.000059,0.000059,0.000059,0.068299,0.000059
2,0.000073,0.957304,0.041969,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073
3,0.007802,0.000067,0.000067,0.000067,0.000067,0.901185,0.000067,0.000067,0.000067,0.000067,0.090413,0.000067
4,0.000074,0.999185,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074
...,...,...,...,...,...,...,...,...,...,...,...,...
987,0.000035,0.000035,0.060902,0.000035,0.000035,0.000035,0.000035,0.189425,0.749361,0.000035,0.000035,0.000035
988,0.000009,0.000009,0.023848,0.441657,0.000009,0.000009,0.000009,0.000009,0.534410,0.000009,0.000009,0.000009
989,0.000085,0.000085,0.278359,0.023831,0.000085,0.000085,0.000085,0.000085,0.697047,0.000085,0.000085,0.000085
990,0.000096,0.000096,0.193870,0.000096,0.000096,0.000096,0.000096,0.048117,0.700366,0.000096,0.056882,0.000096


In [11]:
#Importamos el gravity1.index: 
gravity1=TplGravity(config=config.ds_medium)
# Some rows are small (<5 sec) or unbounded (>30m), this criteria will restrict to only the good ones
gravity1.index = gravity1.index[gravity1.index["SECONDS"] > 5]
gravity1.index = gravity1.index[gravity1.index["SECONDS"] < 1800]
gravity1.index

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,estimated (s),real (s),diff (s),SECONDS
0,2020-12-01 00:16:04.547000+00:00,2020-12-01 00:19:11.750000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,227,307,40,187.0
1,2020-12-01 00:19:11.804000+00:00,2020-12-01 00:23:49.341000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,438,22,277.0
2,2020-12-01 00:23:49.411000+00:00,2020-12-01 00:25:34.908000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,145,7,105.0
3,2020-12-01 00:25:34.967000+00:00,2020-12-01 00:29:33.402000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,359,61,238.0
4,2020-12-01 00:29:33.453000+00:00,2020-12-01 00:31:18.444000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,145,7,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1242,2021-01-01 19:55:07.919000+00:00,2021-01-01 19:59:19.367000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_AcqFibA,False,False,400,412,12,251.0
1243,2021-01-01 19:59:19.458000+00:00,2021-01-01 20:10:17.247000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_AcqFibB,False,False,1125,1058,27,657.0
1245,2021-01-01 20:10:20.661000+00:00,2021-01-01 20:11:58.709000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_PTref,False,False,135,138,3,98.0
1246,2021-01-01 20:11:58.798000+00:00,2021-01-01 20:13:26.902000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_WFSref,False,False,123,128,5,88.0


Con la siguiente función construimos el dataframe final, no tiene otra peculiaridad más que hacer coincidir los índices del gravity1.index con el dataframe topic-trace:

In [30]:
def final_result(obs_index,lda_model,info_vec):
    #obs_index: dataset.index
    #lda_model: bestldamodel
    #info_vec: matrix token-frequency for info
    
    #hacemos coincidir los índices
    final=obs_index
    final.index.name='Num trace'
    final=final.reset_index()
    
    #creamos el dataframe token-trace
    df_topic_dc=ftn.topic_document(lda_model,info_vec)
    #concat
    final_result=pd.concat([final, df_topic_dc], axis=1,)
    return final_result

In [31]:
final_df=final_result(gravity1.index,bestmodel_gravity,gravity_SES_v)
final_df

,Num trace,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,estimated (s),...,2,3,4,5,6,7,8,9,10,11
0,0,2020-12-01 00:16:04.547000+00:00,2020-12-01 00:19:11.750000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,227,...,0.035722,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046
1,1,2020-12-01 00:19:11.804000+00:00,2020-12-01 00:23:49.341000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,...,0.000059,0.000059,0.000059,0.901173,0.000059,0.000059,0.000059,0.000059,0.068299,0.000059
2,2,2020-12-01 00:23:49.411000+00:00,2020-12-01 00:25:34.908000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,...,0.041969,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073,0.000073
3,3,2020-12-01 00:25:34.967000+00:00,2020-12-01 00:29:33.402000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,...,0.000067,0.000067,0.000067,0.901185,0.000067,0.000067,0.000067,0.000067,0.090413,0.000067
4,4,2020-12-01 00:29:33.453000+00:00,2020-12-01 00:31:18.444000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,...,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074,0.000074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,1242,2021-01-01 19:55:07.919000+00:00,2021-01-01 19:59:19.367000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_AcqFibA,False,False,400,...,0.060902,0.000035,0.000035,0.000035,0.000035,0.189425,0.749361,0.000035,0.000035,0.000035
988,1243,2021-01-01 19:59:19.458000+00:00,2021-01-01 20:10:17.247000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_AcqFibB,False,False,1125,...,0.023848,0.441657,0.000009,0.000009,0.000009,0.000009,0.534410,0.000009,0.000009,0.000009
989,1245,2021-01-01 20:10:20.661000+00:00,2021-01-01 20:11:58.709000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_PTref,False,False,135,...,0.278359,0.023831,0.000085,0.000085,0.000085,0.000085,0.697047,0.000085,0.000085,0.000085
990,1246,2021-01-01 20:11:58.798000+00:00,2021-01-01 20:13:26.902000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_WFSref,False,False,123,...,0.193870,0.000096,0.000096,0.000096,0.000096,0.048117,0.700366,0.000096,0.056882,0.000096


In [32]:
#result=open("gravity_result","wb")
#pickle.dump(final_df,result)
#result.close()

In [25]:
final_result=pickle.load(open("gravity_result","rb"))

Perfecto, para terminar re-ejecutaremos todo el modelo  en un nuevo notebook con el fin de ejemplificar y acelerar el proceso de obtención de este resultado, omiteremos el paso a paso y servirá como pauta para futuras implementaciones. 